# Import All Libraries

In [ ]:
# ========== Standard Library Imports ==========
import os
import shutil
import random
from tqdm import tqdm

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress INFO/WARNING logs
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Reduce plugin loading issues

# ========== Data Handling & Visualization ==========
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# ========== Scikit-learn Tools for Evaluation & Preprocessing ==========
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score

# ========== TensorFlow / Keras for Deep Learning ==========
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image

# Load Dataset And Split to (train - test - val)

In [ ]:
# Paths
original_dataset_dir = '/kaggle/input/face-mask-detection/Dataset'
base_dir = '/kaggle/working/Dataset'

# Split ratio
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Create target directories
for split in ['train', 'val', 'test']:
    for class_name in os.listdir(original_dataset_dir):
        os.makedirs(os.path.join(base_dir, split, class_name), exist_ok=True)

# Split and copy files
for class_name in os.listdir(original_dataset_dir):
    class_dir = os.path.join(original_dataset_dir, class_name)
    images = os.listdir(class_dir)
    random.shuffle(images)

    total = len(images)
    train_end = int(train_ratio * total)
    val_end = train_end + int(val_ratio * total)

    train_files = images[:train_end]
    val_files = images[train_end:val_end]
    test_files = images[val_end:]

    for split, split_files in zip(['train', 'val', 'test'], [train_files, val_files, test_files]):
        for fname in tqdm(split_files, desc=f'Copying {split} - {class_name}'):
            src = os.path.join(class_dir, fname)
            dst = os.path.join(base_dir, split, class_name, fname)
            shutil.copyfile(src, dst)


# Data Preprocessing and Augmentation

In [ ]:
# ========== Data Augmentation and Preprocessing ==========
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of training data will be used for validation
)

# Create an ImageDataGenerator for test data with only rescaling
# (no augmentation to keep test data unbiased and realistic)
test_datagen = ImageDataGenerator(rescale=1./255)

# ========== Load and Prepare Training Data ==========
training_set = train_datagen.flow_from_directory(
    '/kaggle/working/Dataset/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # load 80% of the data for training
)

# ========== Load and Prepare Validation Data ==========
validation_set = train_datagen.flow_from_directory(
    '/kaggle/working/Dataset/val',  # ideally should be the same as training directory
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # load 20% for validation
)

# ========== Load and Prepare Test Data ==========
test_set = test_datagen.flow_from_directory(
    '/kaggle/working/Dataset/test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# ========== Display Class Index Mapping ==========
# Print class labels and their corresponding integer indices for each dataset
print("\nTraining Set Classes:", training_set.class_indices)
print("Validation Set Classes:", validation_set.class_indices)
print("Test Set Classes:", test_set.class_indices)

# Model Phase

## Model Architecture

In [ ]:
# Clear previous session to avoid clutter from old models
tf.keras.backend.clear_session()

# Define the CNN model
model = Sequential([
    # 1st Convolutional Block
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),      # Downsample with 2x2 pool size

    # 2nd Convolutional Block
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),      # Downsample with 2x2 pool size

    # 3rd Convolutional Block
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),      # Downsample with 2x2 pool size

    # 4th Convolutional Block
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D(2,2),      # Downsample with 2x2 pool size

    # Fully Connected Layers
    Flatten(),                      # Flatten 2D features to 1D
    Dense(256, activation='relu'),  # Dense layer with 256 units
    Dropout(0.5),                   # Dropout to prevent overfitting
    Dense(training_set.num_classes, activation='softmax') # Output layer for multi-class classification
])

# Compile the model with optimizer, loss function, and evaluation metrics
model.compile(
    optimizer='adam',                        # Adaptive Moment Estimation optimizer
    loss='categorical_crossentropy',         # Suitable for multi-class classification
    metrics=['accuracy']                     # Evaluate using accuracy
)

# Display the model architecture
model.summary()

## Early Stopping

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',        # Watch validation loss
    patience=5,                # Stop after 5 epochs with no improvement
    restore_best_weights=True  # Keep the best model
)

## Class Weights

In [ ]:
# Compute class weights to handle class imbalance during training
# 'balanced' mode automatically gives higher weights to underrepresented classes
class_weights = compute_class_weight(
    class_weight='balanced',                       # Automatically balances based on class frequencies
    classes=np.unique(training_set.classes),       # List of class labels
    y=training_set.classes                         # Actual class labels for all training samples
)

# Convert the result into a dictionary format expected by model.fit()
class_weights = dict(enumerate(class_weights))

## Train the model

In [ ]:
history = model.fit(
              training_set,
              epochs=50,
              validation_data=validation_set,
              class_weight=class_weights,
              callbacks=[early_stop]
          )

# Model Training Metrics

## Evaluate The Model

In [ ]:
loss, accuracy = model.evaluate(test_set)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

## Get Predictions For The Test Set

In [ ]:
y_pred_prob = model.predict(test_set)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = test_set.classes

## Classification Report

In [ ]:
class_labels = list(test_set.class_indices.keys())
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=class_labels))

## Plot Training History

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## Confusion Matrix

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=list(test_set.class_indices.keys()),
            yticklabels=list(test_set.class_indices.keys()))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()

## Receiver Operating Characteristic

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = y_pred_prob.shape[1]  # Get the number of classes

# Binarize the true labels for multi-class ROC calculation
y_true_bin = label_binarize(y_true, classes=np.arange(n_classes))

for i in range(n_classes):
    # Remove the 'label' argument
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curves for each class
plt.figure()
lw = 2  # Line width
colors = ['blue', 'red', 'green']  # Adjust colors if needed

for i in range(n_classes):
    class_label = list(test_set.class_indices.keys())[list(test_set.class_indices.values()).index(i)]  # Getting class label
    # The 'label' argument is used correctly here for the plot legend
    plt.plot(fpr[i], tpr[i], color=colors[i], lw=lw,
             label=f'ROC curve of class {class_label} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

## Precision-Recall Curves

In [ ]:
# Get the actual class names from the test_set.class_indices
class_names = list(test_set.class_indices.keys())

colors = ['blue', 'red', 'green']  # Adjust colors if needed

for i, class_name in enumerate(class_names):
    positive_class_index = i
    y_true_binary = (y_true == positive_class_index).astype(int)
    y_score = y_pred_prob[:, positive_class_index]

    precision, recall, thresholds = precision_recall_curve(y_true_binary, y_score)
    pr_auc = average_precision_score(y_true_binary, y_score)

    plt.plot(recall, precision, color=colors[i], lw=2,
             label=f'{class_name} (PR AUC = {pr_auc:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall Curves for All Classes')
plt.legend(loc="lower left")
plt.show()

# Test Samples

## Predict Image Function

In [ ]:
def predict_image(image_path, model):
    img = image.load_img(image_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction)

    class_labels = list(test_set.class_indices.keys())
    predicted_label = class_labels[predicted_class]

    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Predicted: {predicted_label}')
    plt.show()

    return predicted_label, prediction

## Get Random Image Path

In [ ]:
def get_random_image_path(directory):
    image_files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if not image_files:
      return None  # Handle the case where the directory is empty

    random_image = random.choice(image_files)
    return os.path.join(directory, random_image)

## Mask Weared Incorrect

In [ ]:
mask_incorrect_dir = "/kaggle/working/Dataset/test/mask_weared_incorrect"
random_image_path = get_random_image_path(mask_incorrect_dir)

if random_image_path:
    print(f"Random image from mask_weared_incorrect: {random_image_path}")
    predicted_label, prediction_probabilities = predict_image(random_image_path, model)
    print(f"Predicted Label: {predicted_label}")
    print(f"Prediction Probabilities: {prediction_probabilities}")

else:
    print(f"No images found in {mask_incorrect_dir}")

## With Mask

In [ ]:
with_mask_dir = "/kaggle/working/Dataset/test/with_mask"
random_image_path = get_random_image_path(with_mask_dir)

if random_image_path:
  print(f"Random image from with_mask: {random_image_path}")
  predicted_label, prediction_probabilities = predict_image(random_image_path, model)
  print(f"Predicted Label: {predicted_label}")
  print(f"Prediction Probabilities: {prediction_probabilities}")

else:
  print(f"No images found in {with_mask_dir}")

## Without Mask

In [ ]:
without_mask_dir = "/kaggle/working/Dataset/test/without_mask"
random_image_path = get_random_image_path(without_mask_dir)

if random_image_path:
  print(f"Random image from without_mask: {random_image_path}")
  predicted_label, prediction_probabilities = predict_image(random_image_path, model)
  print(f"Predicted Label: {predicted_label}")
  print(f"Prediction Probabilities: {prediction_probabilities}")

else:
  print(f"No images found in {without_mask_dir}")

# Save Model

In [ ]:
model.save(f'face_mask_detection_model_{accuracy * 100:.2f}acc.h5')